DuckDB 환경 구성하기

In [1]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine

In [2]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [3]:
%sql duckdb:///duckdb.db

Connecting to 'duckdb:///duckdb.db'

In [4]:
%%sql

CREATE SCHEMA IF NOT EXISTS raw_data;
CREATE SCHEMA IF NOT EXISTS analytics;

Running query in 'duckdb:///duckdb.db'

Count


In [5]:
%%sql

CREATE TABLE IF NOT EXISTS raw_data.user_event (
    user_id int,
    datestamp timestamp,
    item_id int,
    clicked int,
    purchased int,
    paidamount int
);

Running query in 'duckdb:///duckdb.db'

Count


In [6]:
%%sql

CREATE TABLE IF NOT EXISTS  raw_data.user_variant (
    user_id int,
    variant_id varchar(32)   -- control vs. test
);

Running query in 'duckdb:///duckdb.db'

Count


In [7]:
%%sql

CREATE TABLE IF NOT EXISTS  raw_data.user_metadata (
    user_id int,
    age varchar(16),
    gender varchar(16)
);

Running query in 'duckdb:///duckdb.db'

Count


In [8]:
%%sql

CREATE TABLE IF NOT EXISTS raw_data.aa_example (
    user_id int,
    date date,
    job_position_id int,
    clicked int,
    checkedout int,
    applied int
);

Running query in 'duckdb:///duckdb.db'

Count


In [9]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/ab_test/user-metadata.csv
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/ab_test/user-event.csv
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/ab_test/user-variant.csv
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/ab_test/aa-example.csv

--2024-12-15 16:19:04--  https://s3-geospatial.s3.us-west-2.amazonaws.com/ab_test/user-metadata.csv
s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com) 해석 중... 52.218.220.217, 52.92.189.82, 52.92.154.154, ...
다음으로 연결 중: s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.218.220.217|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 1820607 (1.7M) [text/csv]
저장 위치: `user-metadata.csv.3'

user-metadata.csv.3 100%[===================>]   1.74M   408KB/s    /  4.4s    

2024-12-15 16:19:09 (408 KB/s) - `user-metadata.csv.3' 저장함 [1820607/1820607]

--2024-12-15 16:19:10--  https://s3-geospatial.s3.us-west-2.amazonaws.com/ab_test/user-event.csv
s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com) 해석 중... 3.5.87.21, 52.92.189.82, 52.92.154.154, ...
다음으로 연결 중: s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|3.5.87.21|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 

In [10]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [13]:
%%sql

INSERT INTO raw_data.user_event 
SELECT * FROM read_csv('user-event.csv', types={'datestamp': 'VARCHAR'});


RuntimeError: (duckdb.duckdb.ConversionException) Conversion Error: CSV Error on Line: 579227
Original Line: 17521,2019-01
Error when converting column "datestamp". Could not convert string "2019-01" to 'TIMESTAMP'

Column datestamp is being converted as type TIMESTAMP
This type was either manually set or derived from an existing table. Select a different type to correctly parse this column.

  file = user-event.csv
  delimiter = , (Auto-Detected)
  quote = " (Auto-Detected)
  escape = " (Auto-Detected)
  new_line = \n (Auto-Detected)
  header = true (Auto-Detected)
  skip_rows = 0 (Auto-Detected)
  comment = \0 (Auto-Detected)
  date_format =  (Auto-Detected)
  timestamp_format =  (Auto-Detected)
  null_padding = 0
  sample_size = 20480
  ignore_errors = false
  all_varchar = 0


[SQL: INSERT INTO raw_data.user_event
SELECT * FROM read_csv('user-event.csv', types={'datestamp': 'VARCHAR'});]
(Background on this error at: https://sqlalche.me/e/14/9h9h)
If you need help solving this issu

In [14]:
%%sql

INSERT INTO raw_data.user_metadata SELECT * FROM 'user-metadata.csv';
INSERT INTO raw_data.user_variant SELECT * FROM 'user-variant.csv';
INSERT INTO raw_data.user_event SELECT * FROM 'user-event.csv';
INSERT INTO raw_data.aa_example SELECT * FROM 'aa-example.csv';

RuntimeError: (duckdb.duckdb.ConversionException) Conversion Error: CSV Error on Line: 579227
Original Line: 17521,2019-01
Error when converting column "datestamp". Could not convert string "2019-01" to 'TIMESTAMP'

Column datestamp is being converted as type TIMESTAMP
This type was auto-detected from the CSV file.
Possible solutions:
* Override the type for this column manually by setting the type explicitly, e.g. types={'datestamp': 'VARCHAR'}
* Set the sample size to a larger value to enable the auto-detection to scan more values, e.g. sample_size=-1
* Use a COPY statement to automatically derive types from an existing table.

  file = user-event.csv
  delimiter = , (Auto-Detected)
  quote = " (Auto-Detected)
  escape = " (Auto-Detected)
  new_line = \n (Auto-Detected)
  header = true (Auto-Detected)
  skip_rows = 0 (Auto-Detected)
  comment = \0 (Auto-Detected)
  date_format =  (Auto-Detected)
  timestamp_format =  (Auto-Detected)
  null_padding = 0
  sample_size = 20480
  ignore_e

In [15]:
%%sql

SELECT *
FROM raw_data.user_event
LIMIT 10

,user_id,datestamp,item_id,clicked,purchased,paidamount


In [16]:
%%sql

SELECT *
FROM raw_data.user_variant
LIMIT 10

,user_id,variant_id
0,0,test
1,1,test
2,2,control
3,3,test
4,4,control
5,5,test
6,6,control
7,7,test
8,8,control
9,9,control


In [17]:
%%sql

SELECT *
FROM raw_data.user_metadata
LIMIT 10

,user_id,age,gender
0,0,50-up,female
1,1,50-up,female
2,2,0-19,female
3,3,50-up,female
4,4,0-19,male
5,5,0-19,female
6,6,0-19,undefined
7,7,50-up,female
8,8,50-up,male
9,9,0-19,male


variant_daily_sessions를 analytics 스키마 밑에 만들기

In [18]:
%%sql

CREATE TABLE analytics.variant_daily_sessions AS
SELECT
    variant_id,
    ue.user_id,
    datestamp,
    count(distinct item_id) num_of_items, -- 총 impression
    sum(clicked) num_of_clicks,                -- 총 click
    sum(purchased) num_of_purchases,  -- 총 purchase
    sum(paidamount) revenue                   -- 총 revenue
FROM raw_data.user_event ue
JOIN raw_data.user_variant uv ON ue.user_id = uv.user_id
GROUP by 1, 2, 3

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name "variant_daily_sessions" already exists!
[SQL: CREATE TABLE analytics.variant_daily_sessions AS
SELECT
    variant_id,
    ue.user_id,
    datestamp,
    count(distinct item_id) num_of_items,
    sum(clicked) num_of_clicks,
    sum(purchased) num_of_purchases,
    sum(paidamount) revenue
FROM raw_data.user_event ue
JOIN raw_data.user_variant uv ON ue.user_id = uv.user_id
GROUP by 1, 2, 3]
(Background on this error at: https://sqlalche.me/e/14/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [19]:
%%sql

SELECT *
FROM analytics.variant_daily_sessions
LIMIT 100

,variant_id,user_id,datestamp,num_of_items,num_of_clicks,num_of_purchases,revenue


In [21]:
con = duckdb.connect('duckdb.db')
# 데이터베이스 작업 수행
con.close()


ConnectionException: Connection Error: Can't open a connection to same database file with a different configuration than existing connections

Two Sample T-Test 실습

In [20]:
duckdb_con = duckdb.connect("duckdb.db")

ConnectionException: Connection Error: Can't open a connection to same database file with a different configuration than existing connections

In [ ]:
variant_daily_sessions = duckdb_con.execute('SELECT * FROM analytics.variant_daily_sessions').df()

In [ ]:
variant_daily_sessions.groupby('variant_id').size()

variant_id
control    82526
test       83116
dtype: int64

In [ ]:
variant_daily_sessions.groupby('variant_id')["revenue"].mean()

variant_id
control    5.590990
test       5.570203
Name: revenue, dtype: float64

## t-score 계산


### scipy.stats.ttest_ind를 이용한 t-score 계산

In [ ]:
from scipy import stats

# control 그룹에 속한 매출액 정보를 numpy 배열로 받아오기
a = variant_daily_sessions[variant_daily_sessions["variant_id"]=="control"]["revenue"].to_numpy()
# test 그룹에 속한 매출액 정보를 numpy 배열로 받아오기
b = variant_daily_sessions[variant_daily_sessions["variant_id"]=="test"]["revenue"].to_numpy()
print(len(a), len(b))

82526 83116


In [ ]:
# control 그룹 매출액 정보 처음 100개 보기
a[0:100]

array([17.,  0., 11.,  0.,  0.,  0., 11., 20.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0., 11.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 13.,  0.,  0.,  0.,
        0.,  0., 19., 37.,  0.,  0.,  0., 41.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., 70.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 37.,  0.,
       48.,  0.,  0., 19.,  0.,  0., 35.,  0.,  0.])

In [ ]:
# test 그룹 매출액 정보 처음 100개 보기
b[0:100]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,  37.,  12.,   0.,   0.,
         0.,   0.,  11.,   0.,  16.,   0.,   0.,   0.,   0.,  11.,   0.,
         0.,   0.,   0.,  37.,   0.,   0.,  11.,   0.,   0.,   0.,  79.,
         0.,   0.,   0.,   0.,   0.,  46.,   0.,  11.,   0.,   0.,   0.,
         0.,   0., 123.,   0.,  23.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  20.,   0.,
         0.,   0.,  11.,   0.,  60.,   0.,  28.,   0.,  13.,   0.,   0.,
         0.,   0.,   0.,  11.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
        11.,  65.,  45.,  20.,   0.,   0.,   0.,   0.,  17.,  11.,   0.,
         0.])

In [ ]:
# ttest_ind 함수를 사용해서 두 그룹의 값들을 비교
# 이 함수는 t-score (사실상 z-score)와 p value를 계산해서 리턴해줌
t, p = stats.ttest_ind(b, a)
print(t, p)

-0.30893666944286313 0.7573700906452349


In [ ]:
# 나중에 사용하기 쉽게 함수로 구현
def compute_z_score(df, field_name):
  a = df[df["variant_id"]=="control"][field_name].to_numpy()
  b = df[df["variant_id"]=="test"][field_name].to_numpy()
  t, p = stats.ttest_ind(b, a)

  return t, p

In [ ]:
print(compute_z_score(variant_daily_sessions, "revenue"))

(-0.30893666944286313, 0.7573700906452349)


### 직접 t-score 계산해보기

 - 그룹별로 크기와 평균과 각 원소 제곱의 합을 알면 계산 가능
   - mean_b와 mean_a (평균)
   - n_b와 n_a (크기)
   - square_b와 square_a (제곱의 평균)
 - 먼저 variance(표준편차의 제곱)은 아래와 같이 계산
   - var_b = square_b - mean_b*mean_b
   - var_a = square_a - mean_a*mean_a
 - 최종 t-score는 아래와 같이 계산
   - (mean_b - mean_a)/math.sqrt(var_b/n_b + var_a/n_a)

In [ ]:
variant_daily_sessions.groupby('variant_id')["revenue"].mean()

variant_id
control    5.590990
test       5.570203
Name: revenue, dtype: float64

In [ ]:
variant_daily_sessions["revenue_2x"] = variant_daily_sessions["revenue"]*variant_daily_sessions["revenue"]

In [ ]:
variant_daily_sessions.head()

,variant_id,user_id,datestamp,num_of_items,num_of_clicks,num_of_purchases,revenue,revenue_2x
0,control,40135,2019-01-14,19,4.0,1.0,17.0,289.0
1,control,40135,2019-01-13,13,3.0,0.0,0.0,0.0
2,control,40135,2019-01-16,2,1.0,1.0,11.0,121.0
3,control,40135,2019-01-15,4,0.0,0.0,0.0,0.0
4,test,89370,2019-01-12,2,1.0,0.0,0.0,0.0


In [ ]:
variant_daily_sessions.groupby('variant_id')["revenue_2x"].mean()

variant_id
control    219.259967
test       217.960044
Name: revenue_2x, dtype: float64

In [ ]:
n_a = 82526
n_b = 83116
mean_a = 22.363958
mean_b = 22.280812
square_a = 3508.159465
square_b = 3487.360701

In [ ]:
import math

var_a = (square_a - mean_a*mean_a)
var_b = (square_b - mean_b*mean_b)
print(var_a, var_b)

3008.0128475742363 2990.926117620656


In [ ]:
t_score = (mean_b - mean_a)/math.sqrt(var_a/n_a+var_b/n_b)
print(t_score)

-0.3089366381664109


In [ ]:
import numpy as np

n_a2 = len(a)
n_b2 = len(b)

mean_a2 = np.mean(a)
mean_b2 = np.mean(b)

var_a2 = np.var(a)
var_b2 = np.var(b)

In [ ]:
t_score2 = (mean_b2 - mean_a2)/math.sqrt(var_a2/n_a2+var_b2/n_b2)
print(t_score2)

-0.30893540027693434


## Impression, Click, Purchase

In [ ]:
# 나중에 사용하기 쉽게 함수로 구현
def compute_z_score(df, field_name):
  a = df[df["variant_id"]=="control"][field_name].to_numpy()
  b = df[df["variant_id"]=="test"][field_name].to_numpy()
  t, p = stats.ttest_ind(b, a)

  return t, p

In [ ]:
# np를 사용해서 계산
def compute_z_score_np(df, field_name):
  a = df[df["variant_id"]=="control"][field_name].to_numpy()
  b = df[df["variant_id"]=="test"][field_name].to_numpy()

  n_a2 = len(a)
  n_b2 = len(b)

  mean_a2 = np.mean(a)
  mean_b2 = np.mean(b)

  var_a2 = np.var(a)
  var_b2 = np.var(b)

  t_score = (mean_b2 - mean_a2)/math.sqrt(var_a2/n_a2+var_b2/n_b2)

  return t_score

In [ ]:
# 모두 직접 계산
def compute_z_score_hand(df, field_name):
  a = df[df["variant_id"]=="control"][field_name].to_numpy()
  b = df[df["variant_id"]=="test"][field_name].to_numpy()

  n_a = len(a)
  n_b = len(b)

  a_sum = 0
  aa = 0
  b_sum = 0
  bb = 0

  for i in a:
    a_sum += i
    aa += i*i

  for i in b:
    b_sum += i
    bb += i*i

  mean_a = a_sum/n_a
  mean_b = b_sum/n_b

  var_a = aa/n_a - mean_a*mean_a
  var_b = bb/n_b - mean_b*mean_b

  t_score = (mean_b - mean_a)/math.sqrt(var_a/n_a+var_b/n_b)

  return t_score

In [ ]:
print(compute_z_score(variant_daily_sessions, 'num_of_items'))
print(compute_z_score_np(variant_daily_sessions, 'num_of_items'))
print(compute_z_score_hand(variant_daily_sessions, 'num_of_items'))

(1.3787392352028343, 0.16797702966025999)
1.3787750204098166
1.3787750204098166


In [ ]:
print(compute_z_score(variant_daily_sessions, 'num_of_clicks'))
print(compute_z_score_np(variant_daily_sessions, 'num_of_clicks'))
print(compute_z_score_hand(variant_daily_sessions, 'num_of_clicks'))

(1.2378079313561117, 0.2157890413600264)
1.2378133780646028
1.2378133780646028


In [ ]:
print(compute_z_score(variant_daily_sessions, 'num_of_purchases'))
print(compute_z_score_np(variant_daily_sessions, 'num_of_purchases'))
print(compute_z_score_hand(variant_daily_sessions, 'num_of_purchases'))

(-0.6220642246344509, 0.5339004941211412)
-0.62205187335538
-0.6220518733553801


All the three z-scores fall in between -1.96 and 1.96; all the p-values are larger than 0.05.

데이터 분석을 위한 요약 테이블 생성

* analytics.analytics_variant_user_daily 생성
* analytics.olap_cube 생성
* analytics.olap_cube의 내용을 'sessions_hypercube.csv'로 저장
* 'sessions_hypercube.csv'를 다운로드

In [ ]:
%%sql

DROP TABLE IF EXISTS analytics.analytics_variant_user_daily;
CREATE TABLE analytics.analytics_variant_user_daily AS
SELECT
  variant_id,
  ue.user_id,
  datestamp, age, gender,
  COUNT(DISTINCT item_id) num_of_items,                             -- 총 impression
  COUNT(DISTINCT CASE WHEN clicked THEN item_id END) num_of_clicks, -- 총 click
  SUM(purchased) num_of_purchases,                                  -- 총 purchase
  SUM(paidamount) revenue                                           -- 총 revenue
FROM raw_data.user_event ue
JOIN raw_data.user_variant uv ON ue.user_id = uv.user_id
JOIN raw_data.user_metadata um ON uv.user_id = um.user_id
GROUP by 1, 2, 3, 4, 5;

,Success


In [ ]:
%%sql

DROP TABLE IF EXISTS analytics.olap_cube;
CREATE TABLE analytics.olap_cube AS
SELECT datestamp, variant_id, age, gender,
    'impression' category,
    count(1) n,    -- number of sessions
    sum(num_of_items) sum, sum(num_of_items*num_of_items) sum2
FROM analytics.analytics_variant_user_daily vds
GROUP BY 1, 2, 3, 4, 5

UNION

SELECT datestamp, variant_id, age, gender,
    'click' category,
    count(1) n,    -- number of sessions
    sum(num_of_clicks) sum, sum(num_of_clicks*num_of_clicks) sum2
FROM analytics.analytics_variant_user_daily vds
GROUP BY 1, 2, 3, 4, 5

UNION

SELECT datestamp, variant_id, age, gender,
    'purchase' category,
    count(1) n,    -- number of sessions
    sum(num_of_purchases) sum, sum(num_of_purchases*num_of_purchases) sum2
FROM analytics.analytics_variant_user_daily vds
GROUP BY 1, 2, 3, 4

UNION

SELECT datestamp, variant_id, age, gender,
    'revenue' category,
    count(1) n,    -- number of sessions
    sum(revenue) sum, sum(revenue*revenue) sum2
FROM analytics.analytics_variant_user_daily vds
GROUP BY 1, 2, 3, 4;

,Success
